In [2]:
import os,sys,string, time
from array import array
import matplotlib.pyplot as plt
import numpy as np
import uproot
import pandas as pd
from platform import python_version
import scipy
from scipy import stats
import uproot3
import math
from matplotlib.patches import Rectangle
import pickle
import functools
from importlib import reload

import Utilities.Plotter as PT
import Utilities.Constants as Constants
import Utilities.Variables_list as Variables
import Utilities.Functions as Functions
from Utilities.ipython_exit import exit

print ('Success')


Success


## Purpose
This notebook takes the input .root file ntuples and creates .pkl files that are used for the rest of the analysis chain.<br> 
These .pkl files are much smaller and easier to deal with in python.<br> 
Which samples will be made into .pkls are defined by the Params dictionary.<br>
This notebook will need to be run at least twice to create all necessary files for the analysis chain; once for "run1" and once for "run3". Always check the names of the samples that are printed and ensure they are what you want to save.

# Setting which samples to pickle

In [3]:
Params = {"Run":"run1", #The run number, so far either "run1" or "run3"
          "Load_lepton_signal":True, #Just e+e- samples, Majorana
          "Load_pi0_signal":False, #Just pi0 samples, Majorana
          "Load_lepton_dirac":False, #Just e+e-, Dirac
          "Load_pi0_dirac":False, #Just pi0, Dirac
          "Load_standard_bkgs":True, #Backgrounds, 'overlay', 'dirtoverlay' and 'beamoff'
          "Load_data":True, #'beamgood' sample
          "Load_DetVars":False, #overlay Detector variations
          "Load_Signal_DetVars":False, #e+e- Detector variation samples
          "Load_pi0_signal_DetVars":False, #pi0 Detector variation samples
          "Only_keep_common_DetVar_evs":True, #For DetVar samples, only keep events common between samples
          "FLATTEN":True, #Have one row per reconstructed object in the dataframe, if True this will effectively make the nslice cut.
          "only_presel":False, #Create small files containing only variables necessary for pre-selection, for making pre-selection plots
          "Load_truth_vars":False, #Load the "mc_" type variables for signal
          "Load_single_file":False, #This will override everything else, put the desired file in the "single_file" line
          "single_file":'beamgood'} 

Params, samples = Functions.new_create_sample_list(Params)
sample_locs_new = Functions.Get_all_sample_locs(Params)

print("Saving these variables: \n" + str(Params["variables_MC"]))


Loading these run1 samples: 
['overlay', 'dirtoverlay', 'beamoff', 'beamgood', '2_ee', '10_ee', '20_ee', '50_ee', '100_ee', '150_ee']
Saving these variables: 
['run', 'sub', 'evt', 'nslice', 'n_pfps', 'n_tracks', 'n_showers', 'swtrig_pre', 'swtrig_post', 'trk_sce_start_x_v', 'trk_sce_start_y_v', 'trk_sce_start_z_v', 'trk_sce_end_x_v', 'trk_sce_end_y_v', 'trk_sce_end_z_v', 'shr_theta_v', 'shr_phi_v', 'shr_px_v', 'shr_py_v', 'shr_pz_v', 'shrclusdir0', 'shrclusdir1', 'shrclusdir2', 'shr_energy_tot', 'trk_theta_v', 'trk_phi_v', 'trk_dir_x_v', 'trk_dir_y_v', 'trk_dir_z_v', 'trk_energy', 'trk_energy_hits_tot', 'trk_energy_tot', 'trk_score_v', 'trk_calo_energy_u_v', 'trk_end_x_v', 'trk_chipr_best', 'pfnplanehits_U', 'pfnplanehits_V', 'pfnplanehits_Y', 'NeutrinoEnergy2', 'SliceCaloEnergy2', 'nu_flashmatch_score', 'contained_sps_ratio', 'flash_time', 'contained_fraction', 'trk_score', 'crtveto', 'shr_tkfit_dedx_U', 'shr_tkfit_dedx_V', 'shr_tkfit_dedx_Y', 'shr_tkfit_dedx_max', 'shr_tkfit_2cm_ded

## Loading and pickling files

In [4]:
common_evs = Functions.Make_common_evs(samples, sample_locs_new, Params)
loc_pkls = "pkl_files/"+Params["Run"]+"/current_files/"+Params["variables_string"]+"/"

Functions.New_load_and_pkl(samples, sample_locs_new, loc_pkls, common_evs, Params, save_str="_full_Finished")

Pickling run1 overlay file


KeyboardInterrupt: 

## End of code

In [ ]:
print(Params["variables_MC"])

In [ ]:
root_dir = 'nuselection'
main_tree = 'NeutrinoSelectionFilter'
loc_pkls = "pkl_files/"+Params["Run"]+"/current_files/"+Params["variables_string"]+"/"

if (Params["Load_DetVars"] == True) and (Params["Only_keep_common_DetVar_evs"] == True):
    df_rse_list = []
    print("Loading only run, subrun, event numbers" + "\n")
    for sample in samples: #Looping over all samples, should make a function for this.
        if sample in Constants.Detector_variations: #Checks if it is an overlay Detector Variation sample
            print(f"Loading overlay {sample} "+Params["Run"]+" with uproot")
            NuMI_MC_overlay = uproot3.open("../NuMI_MC/DetVars/neutrinoselection_filt_"+Params["Run"]+f"_overlay_{sample}.root")[root_dir+"/"+main_tree]
            df_overlay = NuMI_MC_overlay.pandas.df(['run','sub','evt'], flatten=Params["FLATTEN"])
            file = df_overlay
            Functions.make_unique_ev_id(file) #This creates "rse_id" branch
            new_overlay = file.copy()
            df_rse_list.append(new_overlay)

    common_evs = Functions.make_common_evs_df(df_rse_list)
    
elif (Params["Load_Signal_DetVars"] == True) and (Params["Only_keep_common_DetVar_evs"] == True): #WRONG
    common_evs_dict = {}
    print("Loading only run, subrun, event numbers" + "\n")
    if Params["Run"] == "run1": HNL_samples = [150]
    if Params["Run"] == "run3": HNL_samples = [2, 10, 20, 50, 100]
    for HNL_mass in HNL_samples: #Currently don't have 150MeV detvar samples for run3
        df_rse_list = []
        for DetVar in Constants.Detector_variations: #Looping over all samples, should make a function for this.
            #print(f"Loading overlay {sample} "+Params["Run"]+" with uproot")
            NuMI_MC_signal = uproot3.open("../NuMI_signal/KDAR_dump/sfnues/DetVars/"+f"{HNL_mass}_{DetVar}_"+Params["Run"]+".root")[root_dir+"/"+main_tree]
            df_signal = NuMI_MC_signal.pandas.df(['run','sub','evt'], flatten=Params["FLATTEN"])
            file = df_signal
            Functions.make_unique_ev_id(file) #This creates "rse_id" branch
            new_signal = file.copy()
            df_rse_list.append(new_signal)

        common_evs = Functions.make_common_evs_df(df_rse_list)
        common_evs_dict[HNL_mass] = common_evs
        
elif (Params["Load_pi0_signal_DetVars"] == True) and (Params["Only_keep_common_DetVar_evs"] == True): #WRONG
    common_evs_dict = {}
    print("Loading only run, subrun, event numbers" + "\n")
    if Params["Run"] == "run1": HNL_samples = []
    if Params["Run"] == "run3": HNL_samples = [150,180,200,220,240,245]
    for HNL_mass in HNL_samples: #Currently don't have 150MeV detvar samples for run3
        print(HNL_mass)
        df_rse_list = []
        for DetVar in Constants.Detector_variations: #Looping over all samples, should make a function for this.
            #print(f"Loading overlay {sample} "+Params["Run"]+" with uproot")
            NuMI_MC_signal = uproot3.open("../NuMI_signal/KDAR_dump/sfnues/pi0/DetVars/"+f"{HNL_mass}_{DetVar}_"+Params["Run"]+".root")[root_dir+"/"+main_tree]
            df_signal = NuMI_MC_signal.pandas.df(['run','sub','evt'], flatten=Params["FLATTEN"])
            file = df_signal
            Functions.make_unique_ev_id(file) #This creates "rse_id" branch
            new_signal = file.copy()
            df_rse_list.append(new_signal)

        common_evs = Functions.make_common_evs_df(df_rse_list)
        common_evs_dict[HNL_mass] = common_evs
else: common_evs = None

## Making new variables

In [ ]:
def Make_first_new_variables(df): #Only for variables which can be directly made from the ntuple variables with no manipulation/selection
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    #Something like shr_tkfit_distance
    return df.copy

#Works, but takes a while, ignored objects with huge negative numbers for the positions.
def Make_fiducial_vars(df):
    print(len(df))
    n_pfps = df["n_pfps"].groupby(level="entry").apply(max)
    df_placeholder = df.query("trk_sce_start_x_v>-1e4").copy()
    print(len(df_placeholder))
    min_x=df_placeholder[["trk_sce_start_x_v","trk_sce_end_x_v"]].min(axis=1).groupby(level="entry").apply(min)
    max_x=df_placeholder[["trk_sce_start_x_v","trk_sce_end_x_v"]].max(axis=1).groupby(level="entry").apply(max)
    
    min_y=df_placeholder[["trk_sce_start_y_v","trk_sce_end_y_v"]].min(axis=1).groupby(level="entry").apply(min)
    max_y=df_placeholder[["trk_sce_start_y_v","trk_sce_end_y_v"]].max(axis=1).groupby(level="entry").apply(max)
    
    min_z=df_placeholder[["trk_sce_start_z_v","trk_sce_end_z_v"]].min(axis=1).groupby(level="entry").apply(min)
    max_z=df_placeholder[["trk_sce_start_z_v","trk_sce_end_z_v"]].max(axis=1).groupby(level="entry").apply(max)
    
    del df_placeholder
    
    df2 = df.copy()
    
    df2["min_x"]=np.array(np.repeat(min_x, np.array(n_pfps)))
    df2["max_x"]=np.array(np.repeat(max_x, np.array(n_pfps)))
    df2["min_y"]=np.array(np.repeat(min_y, np.array(n_pfps)))
    df2["max_y"]=np.array(np.repeat(max_y, np.array(n_pfps)))
    df2["min_z"]=np.array(np.repeat(min_z, np.array(n_pfps)))
    df2["max_z"]=np.array(np.repeat(max_z, np.array(n_pfps)))
    return df2

# Reading in and pickling files

In [ ]:
if ((Params["Load_Signal_DetVars"] == True)or(Params["Load_pi0_signal_DetVars"] == True)) and (Params["Only_keep_common_DetVar_evs"] == True):
    Functions.Load_and_pkl_samples(samples, sample_loc, loc_pkls, common_evs_dict, Params, save_str="_FINAL")
else:
    Functions.Load_and_pkl_samples(samples, sample_loc, loc_pkls, common_evs, Params, save_str="_FINAL") #Remove "_New_gen" str for data
print("Done!")

# New_load_and_pkl(samples, sample_loc, loc_pkls, common_evs, Params, save_str="")

# Finished code